**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [14]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML

import skvideo
skvideo.setFFmpegPath("C:/Program Files/ffmpeg-4.1-win64-static/bin/")
import skvideo.io

import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Dropout

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4(nb_action) (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act() returns an action depending on the actual state.

With probability $\epsilon$ it will return a random action, otherwise it will return an action computed with the learned policy. Finally, the boolean 'train' allows to ignore the randomness, if the algorithm is not being trained anymore, in which case we would want to play the learned strategy exclusively.

The $\epsilon$ parameter is essential to avoid local minimiser of the Q-value of the policy. In fact, if the algorithm arrives to a policy which never selects a given action, which could seem optimal, the $\epsilon$ parameter allows to play this action randomly regularly. This way, if this action was interesting enough but was not selected in the policy, for any reason, the algorithm will play it anyway, which could enhance the policy.

In a nutshell, it's a way to avoid converging to a local optimizer, by continuing to try some 'random strategy' once in a while.
***

### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

## The environment

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 10 # set small when debugging
epochs_test = 10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The two arrays are of the size of the board, on which had been added two columns and rows on the edges. Thus, for a board of size $10\times10$, the arrays would be of size $14\times14$.

The array `position` contains a 1 one the cell where the rat is, 0 on the other cells of the board and -1 on the cells of the added columns and rows. Thus, 1 means the actual state, 0 an unoccupied cell, and -1 an unreachable cell.

The array `board` contains the value of the reward of each cell of the board: 0.5 if the cell contains a bonus, -1 if it's a malus, and 0 if there is nothing (or if the cell has already been visited).

When a state is given to the algorithm, it contains sub-matrices of those two arrays, of size $5\times5$ centered on the actual position of the rat. Hence, the sub-matrix of `position` gives the information of which cells can be reached, and the sub-matrix of `board` contains the information of the rewards of the near cells.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # We choose a random state between all the possible ones
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a


***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        ### END FILLING
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 12.0/13.0. Average score (-1.0)
Win/lose count 8.0/8.0. Average score (-0.5)
Win/lose count 14.0/23.0. Average score (-3.3333333333333335)
Win/lose count 9.5/18.0. Average score (-4.625)
Win/lose count 12.0/17.0. Average score (-4.7)
Win/lose count 14.5/21.0. Average score (-5.0)
Win/lose count 9.0/17.0. Average score (-5.428571428571429)
Win/lose count 8.5/7.0. Average score (-4.5625)
Win/lose count 12.5/16.0. Average score (-4.444444444444445)
Win/lose count 10.0/16.0. Average score (-4.6)
Final score: -4.6


## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




***
We start from the equation we had previously:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] 
= E_{p^{\pi}}[\gamma^{0}r(s,a) + \sum_{1\leq t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]
= E_{p^{\pi}}[r(s,a) + \gamma \sum_{0\leq t\leq T-1}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a]
= E_{p^{\pi}}[r(s,a) + \gamma \sum_{0\leq t\leq T-1}\gamma^{t}r(s',a')|(s',a')\sim p(.|s,a)] 
= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \> .
\end{equation*}


***
Then, we assume that $\pi^*$ is the optimal policy and exists. Then, it would maximise the rewards, such that, for a state $s'$, the distribution of choosen states $a'$ would maximise the reward. Hence, 
\begin{equation*}
Q^{\pi^*}(s',a') = \max_{a'}Q^{*}(s',a')
\end{equation*}

Thus : 
\begin{equation*}
Q^{\pi*}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi*}(s',a')]\\
= Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

***

The objective of the optimisation will be to maximise $\theta$ in order to get the maximal increased of the Q value from one state to another. Thus, we want to maximise the difference between the optimal $Q(s',a',\theta)$, which is written $\max_{a'}Q(s',a',\theta)$ and $Q(s,a,\theta)$. So we get:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # We add a new element to the list
        self.memory.append(m)
        # If the list is bigger than its max length, we pop the first element
        while len(self.memory) > self.max_memory:
            self.memory.pop(0)
        pass

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 1 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        # MODIFIED :
        # We return the action with the max Q
        prediction_q = self.model.predict([[s]])
        act = np.argmax(prediction_q)

        return act

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        ''' Two steps: first memorize the states, second learn from the pool
        '''

        # Memorization of the current step
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        # Initialization of the training batch
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # We recall a previous state
            s, n_s, a, r, game_over = self.memory.random_access()          

            # We compute the value of target_q for the choosen action 'a'
            if game_over:
                ######## FILL IN
                # If the state is a game_over, it's equal to the reward
                target_q_a = r
                
            else:
                ######## FILL IN
                # Else, it's equal to the reward plus the maximum value of Q of the next state, times gamma
                # The max value of Q_{t+1} is obtained with our current model (Reinforcement Learning)
                prediction_n_s = self.model.predict([[n_s]])
                max_n_q = np.max(prediction_n_s)
                
                target_q_a = r + self.discount * max_n_q

            # We add the values to the X & Y of the training set. The input is the state.
            # The target Q is the currently computed Q for the actions which has not been played, 
            # and the previously updated Q for the action.
            input_states[i] = s
            target_q[i] = self.model.predict([[s]])
            target_q[i][a] = target_q_a
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        
        model = Sequential()
        model.add(Flatten(input_shape = (5,5,self.n_state)))
        model.add(Dense(2048))
        model.add(Dropout(rate=0.25))
        model.add(Dense(512))
        model.add(Dropout(rate=0.25))
#         model.add(Dense(256))
#         model.add(Dropout(rate=0.25))
        model.add(Dense(4))
        
#         Marche pas trop mal : 512 0.25 256 0.25
#         Marche un peu mieux : 1024 .25 512 0.25 256 0.25
#         Marche plutot bien : 2048 .25 512 0.25
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 200, prefix='fc_train')
HTML(display_videos('fc_train9.mp4'))


Epoch 001/200 | Loss 0.0382 | Win/lose count 5.0/5.0 (0.0)
Epoch 002/200 | Loss 0.0325 | Win/lose count 7.0/2.0 (5.0)
Epoch 003/200 | Loss 0.1667 | Win/lose count 5.0/1.0 (4.0)
Epoch 004/200 | Loss 0.0200 | Win/lose count 6.0/0 (6.0)
Epoch 005/200 | Loss 0.0159 | Win/lose count 4.0/3.0 (1.0)
Epoch 006/200 | Loss 0.0196 | Win/lose count 6.0/4.0 (2.0)
Epoch 007/200 | Loss 0.0102 | Win/lose count 9.5/3.0 (6.5)
Epoch 008/200 | Loss 0.0129 | Win/lose count 2.5/2.0 (0.5)
Epoch 009/200 | Loss 0.0108 | Win/lose count 8.5/3.0 (5.5)
Epoch 010/200 | Loss 0.0175 | Win/lose count 5.0/1.0 (4.0)
Epoch 011/200 | Loss 0.0121 | Win/lose count 5.0/2.0 (3.0)
Epoch 012/200 | Loss 0.0092 | Win/lose count 9.0/2.0 (7.0)
Epoch 013/200 | Loss 0.0098 | Win/lose count 13.5/3.0 (10.5)
Epoch 014/200 | Loss 0.0116 | Win/lose count 6.5/0 (6.5)
Epoch 015/200 | Loss 0.0573 | Win/lose count 6.5/2.0 (4.5)
Epoch 016/200 | Loss 0.0075 | Win/lose count 15.0/6.0 (9.0)
Epoch 017/200 | Loss 0.0069 | Win/lose count 11.0/5.0 (6.

Epoch 138/200 | Loss 0.0036 | Win/lose count 9.5/6.0 (3.5)
Epoch 139/200 | Loss 0.0045 | Win/lose count 17.0/1.0 (16.0)
Epoch 140/200 | Loss 0.0067 | Win/lose count 12.0/1.0 (11.0)
Epoch 141/200 | Loss 0.0045 | Win/lose count 15.0/3.0 (12.0)
Epoch 142/200 | Loss 0.0044 | Win/lose count 12.0/1.0 (11.0)
Epoch 143/200 | Loss 0.0520 | Win/lose count 5.0/0 (5.0)
Epoch 144/200 | Loss 0.0026 | Win/lose count 15.0/3.0 (12.0)
Epoch 145/200 | Loss 0.0048 | Win/lose count 24.5/0 (24.5)
Epoch 146/200 | Loss 0.0050 | Win/lose count 13.5/3.0 (10.5)
Epoch 147/200 | Loss 0.0499 | Win/lose count 12.0/5.0 (7.0)
Epoch 148/200 | Loss 0.0583 | Win/lose count 15.5/0 (15.5)
Epoch 149/200 | Loss 0.0029 | Win/lose count 28.5/3.0 (25.5)
Epoch 150/200 | Loss 0.0057 | Win/lose count 18.0/3.0 (15.0)
Epoch 151/200 | Loss 0.0031 | Win/lose count 20.5/4.0 (16.5)
Epoch 152/200 | Loss 0.0050 | Win/lose count 14.0/3.0 (11.0)
Epoch 153/200 | Loss 0.0048 | Win/lose count 14.0/0 (14.0)
Epoch 154/200 | Loss 0.0049 | Win/los

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [32]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        model = Sequential()
        
        model.add(Conv2D(16, (3,3), input_shape = (5,5,self.n_state), activation='relu', strides=(2,2)))
#         model.add(Dropout(rate=0.25))
        print(model.output_shape)
        
#         model.add(MaxPooling2D((2,2)))
#         print(model.output_shape)
        
        model.add(Conv2D(32, (2,2), activation='relu'))
        model.add(Dropout(rate=0.25))
        print(model.output_shape)
        
        model.add(Flatten())
        print(model.output_shape)
        model.add(Dense(32))
        print(model.output_shape)
        model.add(Dense(4))
        print(model.output_shape)

#         Marche bien : 16 (3.3) 0.5 16 (3.3) 0.5 Fl 4
#         Marche bien+ : 16 (2.2) 0.25 16 (2.2) 0.25 Fl 4
#         Marche bien++ : 16 (3.3) 0.25 32 (2.2) 0.25 Fl 4
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
#         model.compile(adam(lr=lr, decay=1e-4), "mse")
        self.model = model

In [20]:
try:
    del agent
except NameError:
    pass

env = Environment(grid_size=size, max_time=200, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,200,prefix='cnn_train')
HTML(display_videos('cnn_train9.mp4'))

(None, 2, 2, 16)
(None, 1, 1, 32)
(None, 32)
(None, 32)
(None, 4)
Epoch 001/200 | Loss 0.0020 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/200 | Loss 0.0040 | Win/lose count 2.0/4.0 (-2.0)
Epoch 003/200 | Loss 0.0106 | Win/lose count 4.0/11.0 (-7.0)
Epoch 004/200 | Loss 0.0012 | Win/lose count 5.5/6.0 (-0.5)
Epoch 005/200 | Loss 0.0098 | Win/lose count 4.0/2.0 (2.0)
Epoch 006/200 | Loss 0.0034 | Win/lose count 3.0/0 (3.0)
Epoch 007/200 | Loss 0.0032 | Win/lose count 4.0/5.0 (-1.0)
Epoch 008/200 | Loss 0.0101 | Win/lose count 6.0/4.0 (2.0)
Epoch 009/200 | Loss 0.0017 | Win/lose count 3.0/7.0 (-4.0)
Epoch 010/200 | Loss 0.0130 | Win/lose count 5.5/3.0 (2.5)
Epoch 011/200 | Loss 0.0005 | Win/lose count 6.5/3.0 (3.5)
Epoch 012/200 | Loss 0.0033 | Win/lose count 5.0/1.0 (4.0)
Epoch 013/200 | Loss 0.0064 | Win/lose count 3.0/1.0 (2.0)
Epoch 014/200 | Loss 0.0077 | Win/lose count 4.0/6.0 (-2.0)
Epoch 015/200 | Loss 0.0029 | Win/lose count 4.5/3.0 (1.5)
Epoch 016/200 | Loss 0.0087 | Win/lose coun

Epoch 138/200 | Loss 0.0051 | Win/lose count 15.0/9.0 (6.0)
Epoch 139/200 | Loss 0.0045 | Win/lose count 12.5/3.0 (9.5)
Epoch 140/200 | Loss 0.0116 | Win/lose count 14.0/4.0 (10.0)
Epoch 141/200 | Loss 0.0534 | Win/lose count 12.0/7.0 (5.0)
Epoch 142/200 | Loss 0.0210 | Win/lose count 5.0/1.0 (4.0)
Epoch 143/200 | Loss 0.0223 | Win/lose count 16.5/9.0 (7.5)
Epoch 144/200 | Loss 0.0104 | Win/lose count 4.5/8.0 (-3.5)
Epoch 145/200 | Loss 0.0096 | Win/lose count 6.0/4.0 (2.0)
Epoch 146/200 | Loss 0.0102 | Win/lose count 15.0/6.0 (9.0)
Epoch 147/200 | Loss 0.0170 | Win/lose count 4.0/3.0 (1.0)
Epoch 148/200 | Loss 0.0015 | Win/lose count 11.0/2.0 (9.0)
Epoch 149/200 | Loss 0.0075 | Win/lose count 12.0/3.0 (9.0)
Epoch 150/200 | Loss 0.0178 | Win/lose count 16.0/2.0 (14.0)
Epoch 151/200 | Loss 0.0026 | Win/lose count 8.0/5.0 (3.0)
Epoch 152/200 | Loss 0.0119 | Win/lose count 16.5/4.0 (12.5)
Epoch 153/200 | Loss 0.0065 | Win/lose count 15.5/5.0 (10.5)
Epoch 154/200 | Loss 0.0081 | Win/lose c

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [21]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

(None, 2, 2, 16)
(None, 1, 1, 32)
(None, 32)
(None, 32)
(None, 4)
Test of the CNN
Win/lose count 14.5/9.0. Average score (5.5)
Win/lose count 5.0/1.0. Average score (4.75)
Win/lose count 8.0/4.0. Average score (4.5)
Win/lose count 5.0/1.0. Average score (4.375)
Win/lose count 9.5/3.0. Average score (4.8)
Win/lose count 7.0/0. Average score (5.166666666666667)
Win/lose count 7.5/1.0. Average score (5.357142857142857)
Win/lose count 5.5/2.0. Average score (5.125)
Win/lose count 10.5/3.0. Average score (5.388888888888889)
Win/lose count 9.0/4.0. Average score (5.35)
Final score: 5.35
Test of the FC
Win/lose count 10.5/4.0. Average score (6.5)
Win/lose count 6.5/1.0. Average score (6.0)
Win/lose count 7.5/2.0. Average score (5.833333333333333)
Win/lose count 21.0/8.0. Average score (7.625)
Win/lose count 17.5/3.0. Average score (9.0)
Win/lose count 15.0/4.0. Average score (9.333333333333334)
Win/lose count 18.0/1.0. Average score (10.428571428571429)
Win/lose count 17.5/1.0. Average score 

In [22]:
HTML(display_videos('cnn_test2.mp4'))

In [23]:
HTML(display_videos('fc_test0.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [34]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 1 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')        
        

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
    
#         The reward is equal to the reward of the case minus the malus if the case has already been visited
        reward = self.board[self.x, self.y]
        if train:
            reward = reward - self.malus_position[self.x, self.y]
        
        self.malus_position[self.x, self.y] = 0.1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        self.malus_position = np.zeros((self.grid_size,self.grid_size))

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state    


In [36]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 200, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore9.mp4'))

(None, 2, 2, 16)
(None, 1, 1, 32)
(None, 32)
(None, 32)
(None, 4)
Epoch 001/200 | Loss 0.0128 | Win/lose count 3.0/22.200000000000045 (-19.200000000000045)
Epoch 002/200 | Loss 0.0075 | Win/lose count 6.0/21.80000000000005 (-15.80000000000005)
Epoch 003/200 | Loss 0.0054 | Win/lose count 1.0/23.300000000000068 (-22.300000000000068)
Epoch 004/200 | Loss 0.0077 | Win/lose count 4.0/22.900000000000063 (-18.900000000000063)
Epoch 005/200 | Loss 0.0383 | Win/lose count 5.5/22.300000000000043 (-16.800000000000043)
Epoch 006/200 | Loss 0.0055 | Win/lose count 2.5/26.700000000000088 (-24.200000000000088)
Epoch 007/200 | Loss 0.0083 | Win/lose count 12.0/26.50000000000006 (-14.50000000000006)
Epoch 008/200 | Loss 0.0068 | Win/lose count 9.5/27.500000000000107 (-18.000000000000107)
Epoch 009/200 | Loss 0.0086 | Win/lose count 7.5/23.30000000000006 (-15.800000000000061)
Epoch 010/200 | Loss 0.0080 | Win/lose count 16.0/23.10000000000005 (-7.100000000000051)
Epoch 011/200 | Loss 0.0111 | Win/lose 

Epoch 093/200 | Loss 0.0285 | Win/lose count 6.5/17.899999999999988 (-11.399999999999988)
Epoch 094/200 | Loss 0.0218 | Win/lose count 17.5/15.199999999999962 (2.300000000000038)
Epoch 095/200 | Loss 0.0322 | Win/lose count 15.0/16.099999999999966 (-1.099999999999966)
Epoch 096/200 | Loss 0.0358 | Win/lose count 18.5/13.09999999999997 (5.4000000000000306)
Epoch 097/200 | Loss 0.0331 | Win/lose count 14.5/15.499999999999964 (-0.9999999999999645)
Epoch 098/200 | Loss 0.0329 | Win/lose count 17.0/18.000000000000007 (-1.000000000000007)
Epoch 099/200 | Loss 0.0306 | Win/lose count 16.5/13.99999999999997 (2.50000000000003)
Epoch 100/200 | Loss 0.0135 | Win/lose count 24.5/13.89999999999998 (10.60000000000002)
Epoch 101/200 | Loss 0.0231 | Win/lose count 12.5/16.899999999999977 (-4.399999999999977)
Epoch 102/200 | Loss 0.0189 | Win/lose count 19.5/14.89999999999997 (4.60000000000003)
Epoch 103/200 | Loss 0.0165 | Win/lose count 19.5/13.999999999999973 (5.500000000000027)
Epoch 104/200 | Loss

Epoch 186/200 | Loss 0.0415 | Win/lose count 13.0/18.700000000000006 (-5.700000000000006)
Epoch 187/200 | Loss 0.0240 | Win/lose count 24.0/22.300000000000033 (1.6999999999999673)
Epoch 188/200 | Loss 0.0396 | Win/lose count 17.5/18.0 (-0.5)
Epoch 189/200 | Loss 0.0256 | Win/lose count 23.5/14.699999999999974 (8.800000000000026)
Epoch 190/200 | Loss 0.0362 | Win/lose count 24.5/19.200000000000024 (5.299999999999976)
Epoch 191/200 | Loss 0.0303 | Win/lose count 14.0/15.099999999999962 (-1.0999999999999623)
Epoch 192/200 | Loss 0.0419 | Win/lose count 19.5/22.600000000000065 (-3.1000000000000654)
Epoch 193/200 | Loss 0.0165 | Win/lose count 19.0/17.900000000000002 (1.0999999999999979)
Epoch 194/200 | Loss 0.0308 | Win/lose count 21.5/16.999999999999982 (4.500000000000018)
Epoch 195/200 | Loss 0.0463 | Win/lose count 20.5/15.899999999999977 (4.600000000000023)
Epoch 196/200 | Loss 0.0381 | Win/lose count 19.5/17.399999999999984 (2.1000000000000156)
Epoch 197/200 | Loss 0.0382 | Win/lose c

In [38]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')


Win/lose count 16.5/8.0. Average score (8.5)
Win/lose count 22.0/5.0. Average score (12.75)
Win/lose count 19.0/10.0. Average score (11.5)
Win/lose count 18.0/8.0. Average score (11.125)
Win/lose count 7.0/0. Average score (10.3)
Win/lose count 11.5/4.0. Average score (9.833333333333334)
Win/lose count 21.0/6.0. Average score (10.571428571428571)
Win/lose count 11.0/4.0. Average score (10.125)
Win/lose count 24.5/9.0. Average score (10.722222222222221)
Win/lose count 24.5/11.0. Average score (11.0)
Final score: 11.0


In [39]:
HTML(display_videos('cnn_test_explore9.mp4'))

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***